In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn import preprocessing
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
biometrics_df = pd.read_csv("../1_standarized_biometrics.csv")
biometrics_df.head()

,HR,HRV,MicroSiemens,ArousalMean
0,-0.255155,-1.609745,-1.658083,0.0
1,0.427759,-1.627232,-1.646859,0.0
2,-0.088221,-1.627232,-1.658083,0.0
3,-0.133748,-1.627232,-1.658083,0.0
4,-0.224803,-1.443622,-1.669307,0.0


In [3]:
X = biometrics_df[['MicroSiemens', 'HR', 'HRV' ]].values
y = biometrics_df['ArousalMean'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

In [5]:
# Para poder usar un clasificador, los datos de la variable target deben ser discretos.
#Los convertimos a True-False (Aroused-NotAroused) con el fin de poder entrear un clasificador binario
def map_to_0_and_1(arousal):
    return 0 if arousal < 5 else 1

y_train_aroused = np.array(list(map(map_to_0_and_1, y_train)))
y_test_aroused = np.array(list(map(map_to_0_and_1, y_test)))

In [6]:
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))
        
        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))
        

In [7]:
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train_aroused)

AdaBoostClassifier(n_estimators=100, random_state=42)

# Prediction

In [8]:
ada_clf.predict(X[145].reshape(1, -1))

array([0])

# TRAIN Performance Measuring

In [9]:
print_score(ada_clf, X_train, X_test, y_train_aroused, y_test_aroused, train=True)
print("\n******************************\n")
print_score(ada_clf, X_train, X_test, y_train_aroused, y_test_aroused, train=False)

Train Result:

accuracy score: 0.7424

Classification Report: 
               precision    recall  f1-score   support

           0       0.77      0.91      0.83      2155
           1       0.62      0.34      0.44       908

    accuracy                           0.74      3063
   macro avg       0.69      0.63      0.64      3063
weighted avg       0.72      0.74      0.72      3063


Confusion Matrix: 
 [[1967  188]
 [ 601  307]]

ROC AUC: 0.6254

Average Accuracy: 	 0.6170
Accuracy SD: 		 0.0687

******************************

Test Result:

accuracy score: 0.5313

Classification Report: 
               precision    recall  f1-score   support

           0       0.52      0.88      0.65       379
           1       0.62      0.19      0.29       387

    accuracy                           0.53       766
   macro avg       0.57      0.53      0.47       766
weighted avg       0.57      0.53      0.47       766


Confusion Matrix: 
 [[335  44]
 [315  72]]

ROC AUC: 0.5350



In [10]:
ada_clf = AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
ada_clf.fit(X_train, y_train_aroused)

AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=100)

In [11]:
print_score(ada_clf, X_train, X_test, y_train_aroused, y_test_aroused, train=True)
print("\n*****************************\n")
print_score(ada_clf, X_train, X_test, y_train_aroused, y_test_aroused, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2155
           1       1.00      1.00      1.00       908

    accuracy                           1.00      3063
   macro avg       1.00      1.00      1.00      3063
weighted avg       1.00      1.00      1.00      3063


Confusion Matrix: 
 [[2155    0]
 [   0  908]]

ROC AUC: 1.0000

Average Accuracy: 	 0.6287
Accuracy SD: 		 0.0697

*****************************

Test Result:

accuracy score: 0.5013

Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.81      0.62       379
           1       0.52      0.20      0.29       387

    accuracy                           0.50       766
   macro avg       0.51      0.50      0.45       766
weighted avg       0.51      0.50      0.45       766


Confusion Matrix: 
 [[306  73]
 [309  78]]

ROC AUC: 0.5045

